In [ ]:
# Lib for 1st phase of the project

%pip install fastapi uvicorn pydantic requests httpx rapidfuzz python-dotenv spacy
%python -m spacy download en_core_web_sm
%pip install google-generativeai


  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached google_generativeai-0.8.5-py3-none-any.whl (155 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# now lets define the type of disasters to rempve any form of ambiguity from the users input

# file: hazard_features.py
HAZARD_FEATURE_MAP = {
    "flood": ["hospitals", "shelters", "evacuation_routes", "road_status",
              "flood_zones", "critical_infra"],
    "cyclone": ["hospitals", "shelters", "evacuation_routes", "road_status",
                "flood_zones", "critical_infra"],
    "earthquake": ["hospitals", "shelters", "road_status", "terrain_profile",
                   "critical_infra"],
    "landslide": ["hospitals", "shelters", "road_status", "terrain_profile",
                  "critical_infra"],
    "wildfire": ["hospitals", "shelters", "evacuation_routes", "road_status",
                 "land_use_land_cover", "critical_infra"],
    "heatwave": ["hospitals", "shelters", "water_points", "critical_infra"],
    "drought": ["hospitals", "water_points", "critical_infra"],
    "industrial_accident": ["hospitals", "shelters", "evacuation_routes",
                            "police_fire", "critical_infra"],
    "unknown": ["hospitals", "shelters", "critical_infra"]
}


In [ ]:
# now we'll use google gemini to parse the user input and extract relevant information
import dotenv
dotenv.load_dotenv()
import json
import os
import google.generativeai as genai

# Get your key from Google AI Studio and set in env
API_KEY = os.getenv("GEMINI_API_KEY")
if not API_KEY:
    raise ValueError("Please set GEMINI_API_KEY in environment.")

genai.configure(api_key=API_KEY)

SYSTEM_PROMPT = """
You are an emergency disaster interpreter for India. 
The user will give you a short text like "Kerala has gotten flooded" or 
"Earthquake hit Guwahati yesterday" or "Mumbai is facing a heatwave".

Your job: Convert it into STRICT JSON with the following keys:

{
  "disaster_type": "...",
  "location_text": "...",
  "pincode": "...",
  "time_horizon": "...",
  "severity_hint": "...",
  "notes": "...",
  "confidence": 0.0
}

- disaster_type must be one of: flood, cyclone, earthquake, landslide, wildfire,
  heatwave, drought, industrial_accident, unknown
- pincode: if the user explicitly mentions or if you can infer a postal PIN code for the area, include it. Otherwise return "".
- time_horizon examples: "now", "next_24h", "next_72h"
- severity_hint: only fill this if the user explicitly uses words like "severe", "mild", "massive", "badly". 
  Do NOT guess severity based only on the disaster type.
- confidence is a float between 0 and 1
- Return ONLY the JSON object, no explanation
"""



model = genai.GenerativeModel("gemini-2.5-flash")  # or gemini-1.5-pro for better quality

def interpret(user_text: str) -> dict:
    response = model.generate_content([SYSTEM_PROMPT, user_text])
    raw = response.text.strip()
    # Extract JSON from response
    start, end = raw.find("{"), raw.rfind("}")
    if start == -1 or end == -1:
        raise ValueError(f"LLM did not return JSON: {raw[:200]}")
    return json.loads(raw[start:end+1])

if __name__ == "__main__":
    user_query=input("Enter a disaster query: ")
    print(interpret(user_query))


{'disaster_type': 'flood', 'location_text': 'Kandivali Lokhandwala area, Mumbai', 'pincode': '400067', 'time_horizon': 'now', 'severity_hint': '', 'notes': '', 'confidence': 1.0}


In [ ]:
from fastapi import FastAPI, HTTPException
from schemas import InterpretedQuery, Phase1Result
from llm import interpret as llm_interpret
from geocode import nominatim_geocode
from hazard_features import HAZARD_FEATURE_MAP
from heuristics import guess_state

app = FastAPI(title="Disaster-AI Phase 1 (Refactored)")

@app.post("/phase1/interpret", response_model=Phase1Result)
def interpret_endpoint(user_query: dict):
    try:
        text = user_query.get("text")
        email = user_query.get("email_for_nominatim")
        data = llm_interpret(text)

        # Fill interpretation
        interp = InterpretedQuery(
            original_text=text,
            disaster_type=data.get("disaster_type","unknown"),
            location_text=data.get("location_text"),
            time_horizon=data.get("time_horizon"),
            severity_hint=data.get("severity_hint"),
            notes=data.get("notes"),
            confidence=float(data.get("confidence",0.6)),
            requested_features=HAZARD_FEATURE_MAP.get(data.get("disaster_type","unknown"), [])
        )

        warnings, geocoded = [], None
        loc_txt = interp.location_text
        if not loc_txt:
            st = guess_state(text)
            if st:
                loc_txt = f"{st}, India"
                warnings.append("location missing; guessed state from text")
        if loc_txt:
            geocoded = nominatim_geocode(loc_txt, email=email)
            if not geocoded:
                warnings.append("geocoder failed to resolve location_text")
        else:
            warnings.append("no location found")

        return Phase1Result(interpretation=interp, geocoded=geocoded, warnings=warnings)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
